***importing packages and libraries***

In [ ]:
import numpy as np
import tensorflow as tf
import os


In [ ]:
def pre_process_image(image_path, target_height):
    # Load image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1)  # Decode to grayscale
    
    # Convert image to float32 and normalize to [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    shape = tf.shape(image)

    # Convert to numpy array
    h, w, _ = shape.numpy()  # or use .tolist()

    # Calculate new width based on target height
    aspect_ratio = tf.cast(w, tf.float32) / tf.cast(h, tf.float32)
    new_width = tf.cast(target_height * aspect_ratio, tf.int32)

    # Resize image
    image_resized = tf.image.resize(image, [target_height, new_width])

    return image_resized

In [ ]:
def augment_image():
    ...

In [ ]:
def input_batch_generator(image_paths, target_height, target_width, batch_size, augmentation_probability=0.2):
    while True:
        batch_images = []
        for image_path in image_paths:
            image = pre_process_image(image_path, target_height)
            if np.random.rand() < augmentation_probability:
                image = augment_image(image)
            if image.shape[1] < target_width:
                pad_width = target_width - image.shape[1]
                image = np.pad(image, ((0, 0), (0, pad_width)), mode='constant', constant_values=1.0)
            else:
                image = image[:, :target_width]
            batch_images.append(image)
            if len(batch_images) == batch_size:
                yield np.array(batch_images)
                batch_images = []